<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim


### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

### Desafío 2
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

### Datos
Como fuente de datos de utilizarán las novelas de "A Song of Ice and Fire" de George Martin. Las mismas fueron obtenidas de Kaggle:
https://www.kaggle.com/datasets/saurabhbadole/game-of-thrones-book-dataset/data

El dataset cotiene cinco archivos de texto, cada uno con un libro de la saga. Se comienza concatenando todos los archivos en un solo corpus:

In [2]:
import os

lista_libros = [
    "1 - A Game of Thrones.txt",
    "2 - A Clash of Kings.txt",
    "3 - A Storm of Swords.txt",
    "4 - A Feast for Crows.txt",
    "5 - A Dance with Dragons.txt"
]

# Crear un corpus en memoria
corpus = ""

for nombre in lista_libros:
    ruta_libro = os.path.join("./Dataset", nombre)
    with open(ruta_libro, "r", encoding="latin-1") as f:
        contenido = f.read()
        corpus += contenido + "\n"  # agrega un salto de línea entre libros



Se muestran las primeras líneas del texto completo, la cantidad de líneas y caracteres:

In [3]:
# Dividir el corpus en líneas
lineas = corpus.splitlines()  # crea una lista de líneas

# Mostrar las primeras 10 líneas
print("Primeras 10 líneas:")
print("\n".join(lineas[:10]))

# Cantidad de líneas
print("\nCantidad de líneas:", len(lineas))

# Cantidad total de caracteres
print("Cantidad total de caracteres:", len(corpus))


Primeras 10 líneas:
A Game Of Thrones 
Book One of A Song of Ice and Fire 
By George R. R. Martin 
PROLOGUE 
"We should start back," Gared urged as the woods began to grow dark around them. "The wildlings are 
dead." 
"Do the dead frighten you?" Ser Waymar Royce asked with just the hint of a smile. 
Gared did not rise to the bait. He was an old man, past fifty, and he had seen the lordlings come and go. 
"Dead is dead," he said. "We have no business with the dead." 
"Are they dead?" Royce asked softly. "What proof have we?" 

Cantidad de líneas: 125666
Cantidad total de caracteres: 9778338


# Preprocesamiento de texto para embeddings

Antes de entrenar embeddings, se aplican varios pasos de limpieza y tokenización del texto:

- **Convertir a minúsculas:**  
  Se unifican todas las palabras para evitar duplicados debidos a mayúsculas/minúsculas.

- **Eliminar encabezados y secciones no deseadas:**  
  Se eliminan nombres de libros, capítulos, autor, prólogo, dedicatoria, tabla de contenidos, notas de cronología y numeración de páginas.

- **Separar en oraciones:**  
  Se divide el texto usando signos de puntuación (`.`, `?`, `!`) para preservar la estructura básica de las oraciones, incluyendo diálogos y frases cortas.

- **Limpiar cada oración:**  
  Se eliminan números y puntuación interna, y se normalizan espacios en blanco para obtener texto limpio.

- **Tokenización y eliminación de *stopwords*:**  
  Cada oración se convierte en una lista de palabras y se eliminan las palabras vacías (como "the", "and", "of") que no aportan significado semántico.


**Nota:** Algunas oraciones resultan muy cortas después de este proceso, especialmente diálogos o frases muy breves. Como alternativa, se evalúa realizar la tokenización por párrafos completos.


In [4]:
import re
import string
from nltk.corpus import stopwords

def preprocesar_texto_por_oracion(text):
    """
    Recibe un texto completo y devuelve una lista de oraciones,
    cada una representada como lista de tokens limpios.
    Esta versión no depende de NLTK punkt.
    """
    # 1. Pasar a minúsculas
    text = text.lower()
    
    # 2. Eliminar encabezados no deseados
    patterns_to_remove = [
        r"a game of thrones",
        r"a clash of kings",
        r"a storm of swords",
        r"a feast for crows",
        r"a dance with dragons",
        r"book [^\n]+",          # líneas tipo "Book One of..."
        r"by george r\. r\. martin",
        r"prologue",
        r"dedication",
        r"contents",
        r"a note on chronology",
        r"a cavil on chronology",
        r"version history.*",
        r"page \d+",             # "Page XX"
    ]
    for pat in patterns_to_remove:
        text = re.sub(pat, " ", text)
    
    # 3. Separar en “oraciones” usando signos de puntuación
    oraciones = re.split(r'[.!?]+', text)
    oraciones = [s.strip() for s in oraciones if s.strip()]
    
    # 4. Tokenizar palabras y limpiar cada oración
    stop_words = set(stopwords.words("english"))
    corpus_tokens = []
    for oracion in oraciones:
        # Eliminar números y puntuación dentro de la oración
        clean_oracion = re.sub(r'\d+', ' ', oracion)
        clean_oracion = clean_oracion.translate(str.maketrans("", "", string.punctuation))
        clean_oracion = re.sub(r'\s+', ' ', clean_oracion).strip()
        
        # Tokenizar por espacios y eliminar stopwords
        tokens = [w for w in clean_oracion.split() if w not in stop_words]
        if tokens:
            corpus_tokens.append(tokens)
    
    return corpus_tokens

# Aplicar al corpus completo
corpus_tokens = preprocesar_texto_por_oracion(corpus)

print("Cantidad de oraciones tokenizadas:", len(corpus_tokens))
print("Ejemplo de primera oración tokenizada:", corpus_tokens[0])



Cantidad de oraciones tokenizadas: 155054
Ejemplo de primera oración tokenizada: ['start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around']


Visualizamos los tokens de las 20 primeras oraciones:

In [5]:
for token in corpus_tokens[:20]:
    print(token)

['start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around']
['wildlings', 'dead']
['dead', 'frighten']
['ser', 'waymar', 'royce', 'asked', 'hint', 'smile']
['gared', 'rise', 'bait']
['old', 'man', 'past', 'fifty', 'seen', 'lordlings', 'come', 'go']
['dead', 'dead', 'said']
['business', 'dead']
['dead']
['royce', 'asked', 'softly']
['proof']
['saw', 'gared', 'said']
['says', 'dead', 'thats', 'proof', 'enough']
['known', 'would', 'drag', 'quarrel', 'sooner', 'later']
['wished', 'later', 'rather', 'sooner']
['mother', 'told', 'dead', 'men', 'sing', 'songs', 'put']
['wet', 'nurse', 'said', 'thing', 'royce', 'replied']
['never', 'believe', 'anything', 'hear', 'womans', 'tit']
['things', 'learned', 'even', 'dead']
['voice', 'echoed', 'loud', 'twilit', 'forest']


#### Alternativa: Tokenizar por párrafos

In [6]:
import re
import string
from nltk.corpus import stopwords

def preprocesar_texto_por_parrafo(text):
    """
    Recibe un texto completo y devuelve una lista de párrafos,
    cada uno representado como lista de tokens limpios.
    """
    # 1. Pasar a minúsculas
    text = text.lower()

    # 2. Reemplazar caracteres especiales comunes
    text = text.replace("\x97", " ")  # guion largo
    text = text.replace("\x96", " ")  # guion corto
    text = text.replace("\x91", "'").replace("\x92", "'")  # comillas simples
    text = text.replace("\x93", '"').replace("\x94", '"')  # comillas dobles
    # eliminar cualquier otro caracter no ASCII
    text = re.sub(r'[^\x00-\x7F]+', ' ', text)
    
    # 3. Eliminar encabezados no deseados
    patterns_to_remove = [
        r"a game of thrones",
        r"a clash of kings",
        r"a storm of swords",
        r"a feast for crows",
        r"a dance with dragons",
        r"book [^\n]+",
        r"by george r\. r\. martin",
        r"prologue",
        r"dedication",
        r"contents",
        r"a note on chronology",
        r"a cavil on chronology",
        r"version history.*",
        r"page \d+",
    ]
    for pat in patterns_to_remove:
        text = re.sub(pat, " ", text)
    
    # 4. Separar en párrafos usando saltos de línea
    parrafos = text.split("\n")
    parrafos = [p.strip() for p in parrafos if p.strip()]
    
    # 5. Limpiar cada párrafo y tokenizar
    stop_words = set(stopwords.words("english"))
    corpus_tokens = []
    for parrafo in parrafos:
        # Eliminar números y puntuación
        clean_parrafo = re.sub(r'\d+', ' ', parrafo)
        clean_parrafo = clean_parrafo.translate(str.maketrans("", "", string.punctuation))
        clean_parrafo = re.sub(r'\s+', ' ', clean_parrafo).strip()
        
        # Tokenizar por espacios y eliminar stopwords
        tokens = [w for w in clean_parrafo.split() if w not in stop_words]
        if tokens:
            corpus_tokens.append(tokens)
    
    return corpus_tokens

# Aplicar al corpus completo
corpus_tokens_parrafo = preprocesar_texto_por_parrafo(corpus)

print("Cantidad de párrafos tokenizados:", len(corpus_tokens_parrafo))
print("Ejemplo de primer párrafo tokenizado:", corpus_tokens_parrafo[0])

for token in corpus_tokens_parrafo[:20]:
    print(token)


Cantidad de párrafos tokenizados: 118164
Ejemplo de primer párrafo tokenizado: ['start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around', 'wildlings']
['start', 'back', 'gared', 'urged', 'woods', 'began', 'grow', 'dark', 'around', 'wildlings']
['dead']
['dead', 'frighten', 'ser', 'waymar', 'royce', 'asked', 'hint', 'smile']
['gared', 'rise', 'bait', 'old', 'man', 'past', 'fifty', 'seen', 'lordlings', 'come', 'go']
['dead', 'dead', 'said', 'business', 'dead']
['dead', 'royce', 'asked', 'softly', 'proof']
['saw', 'gared', 'said', 'says', 'dead', 'thats', 'proof', 'enough']
['known', 'would', 'drag', 'quarrel', 'sooner', 'later', 'wished', 'later', 'rather']
['sooner', 'mother', 'told', 'dead', 'men', 'sing', 'songs', 'put']
['wet', 'nurse', 'said', 'thing', 'royce', 'replied', 'never', 'believe', 'anything', 'hear', 'womans']
['tit', 'things', 'learned', 'even', 'dead', 'voice', 'echoed', 'loud', 'twilit', 'forest']
['long', 'ride', 'us', 'gared', 'pointed', 'eight',

### Entrenamiento de modelos de embeddings:

Agregamos la función de callback vista en clase para informar el loss de cada época:

In [7]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

Entrenamos dos variantes del modelo, tanto con **CBOW** como con **Skipgram**:

Si bien en el ejemplo en clase entrenamos embeddings con dimensión 300, dado que observamos que las oraciones son bastante cortas, disminuimos la dimensionalidad del embedding a 100

#### Entrenamiento modelo Skipgram

In [8]:
# Crearmos el modelo generador de vectores
# En este caso utilizaremos la estructura modelo Skipgram
w2v_model_sg = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=100,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=5,      # si tienen más cores pueden cambiar este valor
                     sg=1)           # modelo 0:CBOW  1:skipgram

In [9]:
w2v_model_sg.build_vocab(corpus_tokens)

# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", w2v_model_sg.corpus_count)

# Cantidad de palabras encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(w2v_model_sg.wv.index_to_key))

Cantidad de docs en el corpus: 155054
Cantidad de words distintas en el corpus: 12698


In [10]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
w2v_model_sg.train(corpus_tokens,
                 total_examples=w2v_model_sg.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 1710212.5
Loss after epoch 1: 1300642.25
Loss after epoch 2: 1220713.75
Loss after epoch 3: 1099945.5
Loss after epoch 4: 1030939.5
Loss after epoch 5: 1068403.5
Loss after epoch 6: 1006568.0
Loss after epoch 7: 941937.0
Loss after epoch 8: 930343.0
Loss after epoch 9: 972933.0
Loss after epoch 10: 916532.0
Loss after epoch 11: 901545.0
Loss after epoch 12: 943384.0
Loss after epoch 13: 891947.0
Loss after epoch 14: 849576.0
Loss after epoch 15: 973904.0
Loss after epoch 16: 784144.0
Loss after epoch 17: 855848.0
Loss after epoch 18: 822212.0
Loss after epoch 19: 761792.0
Loss after epoch 20: 807646.0
Loss after epoch 21: 850472.0
Loss after epoch 22: 804488.0
Loss after epoch 23: 848590.0
Loss after epoch 24: 794140.0
Loss after epoch 25: 830432.0
Loss after epoch 26: 843522.0
Loss after epoch 27: 826294.0
Loss after epoch 28: 833718.0
Loss after epoch 29: 793918.0
Loss after epoch 30: 784144.0
Loss after epoch 31: 796608.0
Loss after epoch 32: 728536.0
Loss after 

(44091254, 47474250)

#### Entrenamiento modelo CBOW:

In [11]:
w2v_model_cbow = Word2Vec(min_count=5,    # frecuencia mínima de palabra para incluirla en el vocabulario
                     window=2,       # cant de palabras antes y desp de la predicha
                     vector_size=100,       # dimensionalidad de los vectores 
                     negative=20,    # cantidad de negative samples... 0 es no se usa
                     workers=5,      # si tienen más cores pueden cambiar este valor
                     sg=0)           # modelo 0:CBOW  1:skipgram

In [12]:
w2v_model_cbow.build_vocab(corpus_tokens)

w2v_model_cbow.train(corpus_tokens,
                 total_examples=w2v_model_cbow.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 869420.9375
Loss after epoch 1: 620627.8125
Loss after epoch 2: 565155.25
Loss after epoch 3: 490026.75
Loss after epoch 4: 494456.5
Loss after epoch 5: 488458.75
Loss after epoch 6: 433861.5
Loss after epoch 7: 440861.5
Loss after epoch 8: 398077.0
Loss after epoch 9: 406698.0
Loss after epoch 10: 373864.0
Loss after epoch 11: 404653.5
Loss after epoch 12: 402789.5
Loss after epoch 13: 389241.0
Loss after epoch 14: 389332.5
Loss after epoch 15: 373365.5
Loss after epoch 16: 371654.5
Loss after epoch 17: 376084.0
Loss after epoch 18: 368897.5
Loss after epoch 19: 369510.0
Loss after epoch 20: 365713.0
Loss after epoch 21: 372257.0
Loss after epoch 22: 377429.0
Loss after epoch 23: 383292.0
Loss after epoch 24: 380198.0
Loss after epoch 25: 372340.0
Loss after epoch 26: 379201.0
Loss after epoch 27: 371796.0
Loss after epoch 28: 369310.0
Loss after epoch 29: 364093.0
Loss after epoch 30: 366635.0
Loss after epoch 31: 355112.0
Loss after epoch 32: 357708.0
Loss after 

(44090564, 47474250)

### Análisis de resultados

Evaluamos las 10 más palabras más similares, y las 10 menos similares, para distintos personajes, apellidos o palabras importantes de las novelas. En cada caso evaluamos tanto los modelos de CBOW como Skipgram:

In [13]:
def mostrar_similares(model, palabra, topn=10, tipo_modelo= ""):
    """
    Muestra una tabla con los n más similares (positivos)
    y los n menos similares (negativos) a la palabra dada.
    """
    positivos = model.wv.most_similar(positive=[palabra], topn=topn)
    negativos = model.wv.most_similar(negative=[palabra], topn=topn)

    # Convertir en DataFrames
    df_pos = pd.DataFrame(positivos, columns=["Positivo", "Similitud"])
    df_neg = pd.DataFrame(negativos, columns=["Negativo", "Similitud"])

    # Combinar lado a lado
    tabla = pd.concat([df_pos, df_neg], axis=1)

    print(f"\nPalabra consultada: {palabra}\n")
    if tipo_modelo:
        print(f"\Modelo utilizado: {tipo_modelo}\n")
    print(tabla.to_string(index=False))

In [ ]:
mostrar_similares(w2v_model_sg, "tyrion", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "tyrion", 10, "CBOW")




Palabra consultada: tyrion

\Modelo utilizado: Skipgram

 Positivo  Similitud   Negativo  Similitud
   cersei   0.739829    fishing   0.102873
    jaime   0.713796 surrounded   0.058447
    dwarf   0.674522    trained   0.049203
    kevan   0.605468   southron   0.046657
crookedly   0.596174      andal   0.041858
  brienne   0.592341       the   0.035507
   alayne   0.588232      badge   0.034745
     dany   0.587753     mostly   0.030968
    sansa   0.581602      swift   0.028459
  catelyn   0.561408  attacking   0.022322

Palabra consultada: tyrion

\Modelo utilizado: CBOW

Positivo  Similitud    Negativo  Similitud
   jaime   0.798096      called   0.412795
  cersei   0.751257       knows   0.402898
    dany   0.700668        plus   0.386118
 brienne   0.669359        the   0.384119
   dwarf   0.668421      chiefs   0.381825
   sansa   0.604185   consigned   0.376096
   theon   0.592169     emerged   0.373043
    shae   0.590458   smothered   0.368286
    arya   0.588943    infes

#### Similitud con palaba **Tyrion**
- **Skipgram**:
  - **Positivos**: Captura lazos familiares Lannister ("cersei" 0.74, "jaime" 0.69) y el rasgo distintivo "dwarf" (0.64). Incluye personajes narrativos clave como "dany", "brienne", "sansa" (~0.57-0.59) y términos de intriga como "varys" y "alayne".
  - **Negativos**: Términos irrelevantes como "fishing", "nests" (<0.13).
  - **Patrones**: Enfatiza relaciones específicas (familia, rasgos físicos) y tramas de intriga.
- **CBOW**:
  - **Positivos**: Prioriza relaciones familiares ("jaime" 0.78, "cersei" 0.75) y conexiones narrativas más amplias con Stark ("catelyn", "ned", "arya" ~0.59-0.61) y "griff" (0.55).
  - **Negativos**: Términos genéricos como "called", "knows" (~0.36-0.42), menos específicos.
  - **Patrones**: Captura contextos narrativos más generales, incluyendo conflictos Stark-Lannister.

In [26]:
mostrar_similares(w2v_model_sg, "stark", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "stark", 10, "CBOW")


Palabra consultada: stark

\Modelo utilizado: Skipgram

Positivo  Similitud  Negativo  Similitud
  starks   0.674441   barrels   0.077658
  eddard   0.623175      jars   0.070255
 starks   0.606944  unwashed   0.062891
   tully   0.586711  cookfire   0.058753
    robb   0.572691   hopping   0.057505
leobalds   0.554411     necks   0.052534
eddards   0.547003  serpents   0.050519
karstark   0.524763 mollander   0.050251
direwolf   0.520917 onehanded   0.048464
 brandon   0.517103  bursting   0.047685

Palabra consultada: stark

\Modelo utilizado: CBOW

   Positivo  Similitud      Negativo  Similitud
     starks   0.682775       grasses   0.435065
    starks   0.616588          bees   0.408846
      tully   0.487811       firepit   0.397636
     father   0.469191 diamondshaped   0.394681
grandfather   0.452066       incense   0.391934
   karstark   0.448339           dew   0.390502
    greyjoy   0.443791        slices   0.387929
 winterfell   0.442422         whale   0.387268
       

#### Similitud con palaba **Stark**
- **Skipgram**:
  - **Positivos**: Enfatiza la familia ("eddard" 0.65, "robb" 0.59) y el hogar ("winterfell" 0.53), con la alianza Tully ("tully" 0.54) y personajes secundarios ("leobalds", "daryn" ~0.51-0.52).
  - **Negativos**: Términos irrelevantes como "unwashed", "onions" (<0.07).
  - **Patrones**: Destaca la identidad nuclear de los Stark y su conexión geográfica.
- **CBOW**:
  - **Positivos**: Captura formas derivadas ("starks" 0.69, "stark’s" 0.63), alianzas ("tully" 0.50), y términos familiares ("father", "grandfather" ~0.45-0.47), con conexiones narrativas ("greyjoy", "robert" ~0.44).
  - **Negativos**: Términos genéricos como "grasses", "firepit" (~0.39-0.45), menos específicos.
  - **Patrones**: Enfatiza roles familiares y conexiones históricas más amplias.

In [19]:
mostrar_similares(w2v_model_sg, "dragon", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "dragon", 10, "CBOW")


Palabra consultada: dragon

\Modelo utilizado: Skipgram

   Positivo  Similitud Negativo  Similitud
threeheaded   0.589148     luke   0.104582
   dragons   0.585153  glances   0.093567
    dragons   0.575616   orphan   0.091147
   daenerys   0.553533     hugh   0.089918
    dynasty   0.550802  collect   0.088536
    viserys   0.546775     kyle   0.083490
  stormborn   0.520053   feeble   0.081725
 targaryens   0.508092    cloud   0.078050
  targaryen   0.503702    mudge   0.077691
     vhagar   0.498106 probably   0.075495

Palabra consultada: dragon

\Modelo utilizado: CBOW

 Positivo  Similitud   Negativo  Similitud
  dragons   0.500124     cuffed   0.401113
   drogon   0.460943   thrummed   0.397417
conqueror   0.441674   supplied   0.384669
    lions   0.432969      toyed   0.371589
    bitch   0.408729   observed   0.352525
 dragons   0.400938     combed   0.348025
     fire   0.397324  affection   0.347497
  meraxes   0.387619    snoring   0.343151
conquerer   0.386962 hungerf

#### Similitud con palaba **Dragon**
- **Skipgram**:
  - **Positivos**: Captura el simbolismo Targaryen ("threeheaded" 0.62, "stormborn" 0.50) y nombres de dragones ("drogon" 0.56) y figuras históricas ("visenya", "aegon" ~0.49-0.51).
  - **Negativos**: Términos irrelevantes como "greyfaced", "mudge" (<0.10).
  - **Patrones**: Enfoca la mitología Targaryen y términos específicos de dragones.
- **CBOW**:
  - **Positivos**: Incluye dragones ("dragons" 0.54, "drogon" 0.41, "meraxes" 0.39, "balerion" 0.38) y temas de conquista ("conqueror" 0.43, "targaryen" 0.39), pero con ruido como "bitch" (0.41).
  - **Negativos**: Términos genéricos como "terrance", "scowled" (~0.36-0.40), menos informativos.
  - **Patrones**: Captura temas de conquista y linaje, pero con menor especificidad.

In [20]:
mostrar_similares(w2v_model_sg, "throne", 10, "Skipgram")
mostrar_similares(w2v_model_cbow, "throne", 10, "CBOW")


Palabra consultada: throne

\Modelo utilizado: Skipgram

  Positivo  Similitud  Negativo  Similitud
      iron   0.566498    popped   0.116494
     crown   0.549585    emrick   0.108473
     chair   0.540312     arron   0.091908
birthright   0.530507     tells   0.082881
 fidgeting   0.517336      rast   0.071709
rightfully   0.509013  recruits   0.071672
   derives   0.501733 direction   0.070048
     hinge   0.501473    mounds   0.068932
  rightful   0.498886      bill   0.068045
   joffrey   0.494009     fever   0.062494

Palabra consultada: throne

\Modelo utilizado: CBOW

  Positivo  Similitud   Negativo  Similitud
     chair   0.526427       lice   0.388044
     crown   0.492496    buttery   0.358328
   victory   0.480844      piney   0.358166
      holt   0.453929      shoot   0.357907
      seat   0.452341     combed   0.352613
   thrones   0.439575 disturbing   0.349671
  damnable   0.437091      watty   0.346314
   sconces   0.413989    weaving   0.339432
birthright   0.4099

#### Similitud con palaba **throne**
- **Skipgram**:
  - **Positivos**: Destaca el simbolismo del Trono de Hierro ("iron" 0.55, "chair" 0.54, "crown" 0.52) y términos de legitimidad ("rightfully", "birthright" ~0.49-0.50).
  - **Negativos**: Términos irrelevantes como "messages", "pies" (<0.11).
  - **Patrones**: Enfoca la conexión física y simbólica del trono con el poder.
- **CBOW**:
  - **Positivos**: Captura sinónimos ("crown" 0.50, "chair" 0.48, "seat" 0.44) y temas de conquista ("victory" 0.46, "birthright" 0.41), pero incluye ruido como "sconces" (0.41).
  - **Negativos**: Términos genéricos como "combed", "stammered" (~0.33-0.36), menos específicos.
  - **Patrones**: Enfatiza temas amplios de realeza y conquista, pero con menor precisión.

## Diferencias entre Skipgram y CBOW

1. **Especificidad vs. Generalización**:
   - **Skipgram**: Sobresale en capturar relaciones específicas y términos raros, como nombres propios ("qhorin", "ygritte", "visenya"), rasgos distintivos ("dwarf"), y símbolos precisos ("threeheaded", "iron"). Esto lo hace ideal para contextos narrativos únicos y personajes o conceptos con características distintivas en el corpus.
   - **CBOW**: Tiende a generalizar, capturando temas más amplios (e.g., "father", "victory") y conexiones narrativas extensas (e.g., "davos", "greyjoy"). Sin embargo, incluye términos menos relevantes (e.g., "bitch", "sconces") y negativos más genéricos (e.g., "called", "grasses"), lo que reduce su precisión para detalles específicos.

2. **Similitudes en Positivos**:
   - **Skipgram**: Produce similitudes ligeramente más altas para términos muy específicos (e.g., "cersei" 0.74 para Tyrion, "threeheaded" 0.62 para dragon) y prioriza relaciones cercanas (familia, lugares, rasgos).
   - **CBOW**: Genera similitudes más altas para relaciones familiares o temáticas amplias (e.g., "jaime" 0.78 para Tyrion, "starks" 0.69 para stark), pero incluye términos menos relevantes, indicando una captura más general del contexto.

3. **Calidad de Negativos**:
   - **Skipgram**: Los negativos son más específicos e irrelevantes (e.g., "fishing", "sceptre" <0.13), lo que demuestra una mejor diferenciación de contextos no relacionados.
   - **CBOW**: Los negativos son más genéricos (e.g., "called", "quailed" ~0.33-0.45), lo que refleja su tendencia a asociar términos comunes en diálogos o descripciones, reduciendo la claridad del análisis.

4. **Robustez ante Ruido**:
   - **Skipgram**: Menos propenso a incluir términos irrelevantes, pero afectado por artefactos de preprocesamiento (e.g., “aegon”, “jon”).
   - **CBOW**: Más propenso a ruido (e.g., "bitch" para dragon, "sconces" para throne), lo que sugiere que un umbral de frecuencia mayor (e.g., `min_count=10`) podría mejorar los resultados.

5. **Contexto Narrativo**:
   - **Skipgram**: Captura mejor el "sabor" de la narrativa, destacando personajes secundarios (e.g., "qhorin", "leobalds") y detalles simbólicos (e.g., "threeheaded", "iron"), ideales para análisis de arcos específicos.
   - **CBOW**: Mejor para temas generales, como liderazgo ("victory"), familia ("father"), y rivalidades ("lions"), pero menos efectivo para detalles específicos.

---

#### Observaciones Generales
- **Patrones Narrativos Comunes**:
  - Ambos modelos capturan relaciones familiares (Lannister para "tyrion", Stark para "jon" y "stark"), geográficas ("winterfell" para stark), mitológicas ("drogon", "targaryen" para dragon), y simbólicas ("iron", "crown" para throne).
  - Los resultados reflejan la narrativa de *A Song of Ice and Fire*: conflictos familiares (Stark vs. Lannister), poder (trono, dragones), y lealtades (Jon con la Guardia).


**Conclusión**: Skipgram es superior para capturar detalles específicos y términos raros. CBOW destaca en temas generales y conexiones narrativas amplias, pero es menos preciso debido a su tendencia a incluir ruido. Ambos modelos son complementarios para analizar la semántica de *A Song of Ice and Fire*, con Skipgram ofreciendo mayor precisión y CBOW mayor cobertura temática.

### Visualizar agrupación de vectores

In [21]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [22]:
# Graficar los embedddings en 2D
import plotly.graph_objects as go
import plotly.express as px

vecs, labels = reduce_dimensions(w2v_model_sg)



In [23]:
MAX_WORDS=200
fig = px.scatter(x=vecs[:MAX_WORDS,0], y=vecs[:MAX_WORDS,1], text=labels[:MAX_WORDS])
fig.show(renderer="colab") # esto para plotly en colab

In [24]:
vecs_3d, labels_3d = reduce_dimensions(w2v_model_sg, 3)

In [25]:
fig = px.scatter_3d(x=vecs_3d[:MAX_WORDS,0], y=vecs_3d[:MAX_WORDS,1], z=vecs_3d[:MAX_WORDS,2],text=labels_3d[:MAX_WORDS])
fig.update_traces(marker_size = 2)
fig.show(renderer="colab") # esto para plotly en colab

### Análisis del gráfico
Se observa que en 3D compacta la gran mayoría de las palabras en una región, con algunas palabras sueltas, por lo que analizamos sobre el gráfico 2D que provee mayor claridad. Para facilitar el análisis, tomamos capturas del gráfico:


<img src="Capturas T-SNE/Captura_1.png" alt="imagen1" style="width: 100%;">

Se observa una relación entre familiares que son Tyrion, Jaime y Cersei, observada previamente en el análisis de similitud. También Cersei es la más cercana a la palabra "queen", siendo que ella se convierte en reina durante la saga.

<img src="Capturas T-SNE/Captura_2.png" alt="imagen1" style="width: 100%;">

Se observa una relación entre familiares (Stannis y Robert), y la posición que ocupan (Robert es rey, Stannis principe y aspirante a rey)

<img src="Capturas T-SNE/Captura_3.png" alt="imagen1" style="width: 100%;">

Se observa que logra captar relaciones entre colores, además de que aparece la palabra "hair" como cercana

<img src="Capturas T-SNE/Captura_4.png" alt="imagen1" style="width: 100%;">

Logra identificar como cercanas palabras de relaciones familiares como "hija", "hijo", como "casa" (en el sentido de familia)

<img src="Capturas T-SNE/Captura_5.png" alt="imagen1" style="width: 100%;">

Identifica los números 2 y 3 como cercanos

<img src="Capturas T-SNE/Captura_6.png" alt="imagen1" style="width: 100%;">

Identifica como cercanos palabras como "stark", "ned" (miembro de esa familia), "robb" (hijo de Ned), "winterfell" (hogar de los Stark), y también aparece como cercana "lannister" (que es la familia en conflicto con los Stark)

<img src="Capturas T-SNE/Captura_7.png" alt="imagen1" style="width: 100%;">

Las facciones de los "other", los hombres ("men"), y "children" (en referencia a Children of the Forest), estuvieron en guerra en las leyendas narradas en las novelas, por lo que las identifica como cercanas.

### Conclusiones
1. Es fundamental el preprocesamiento de texto para entrenar los modelos. Si bien se eliminaron acentos, stopwords, y caracteres especiales, al momento de capturar las similitudes aparecieron algunos caracteres restantes para limpiar.
2. Skipgram captura relaciones específicas y términos raros (e.g., "qhorin", "threeheaded") al predecir el contexto a partir de una palabra central, siendo ideal para detalles narrativos, pero sensible a artefactos de preprocesamiento. CBOW generaliza temas amplios (e.g., "father", "victory") al predecir una palabra a partir de su contexto, pero incluye más ruido (e.g., "bitch") y negativos genéricos, requiriendo un filtrado más estricto.
3. Los modelos en general capturaron como mayor similitud las relaciones familiares (por ejemplo "Tyrion", "Jaime" y "Cersei", o "Ned" y "Robb").
4. Aparecieron también adjetivos muy relacionados a un determinado personaje, por ejemplo la palabra "enano" ("dwarf") con "Tyrion".
5. En el caso de palabras más generales como "throne" o "dragon", logró capturar tanto su relación con personajes, como su pertenencia a símbolos de casas, o a otros conceptos relacionados (por ejemplo "birthright" o "rightfully" con "throne")
6. Por último, también logra capturar palabras similares por fuera del contexto de la novela (en los ejemplos mostrados, aparecieron colores y números)